In [4]:
%matplotlib widget

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
import pickle
import pandas as pd

In [5]:
#parameters
scan_id=3
sample_rate=10
no_of_lines =50
line_tolarance=1.5
model_name="svm"

In [6]:
def process_data(file, sample_rate = sample_rate):
    coord = []
    for point in file.readlines():
        coord.append(list(map(float,point.split())))
        
    point_array_sampled = []
    for i in range(0,len(coord),sample_rate):
        point_array_sampled.append(coord[i])
        
    point_array_sampled = np.array(point_array_sampled)
        
    return np.array( point_array_sampled)

In [7]:
def plot_graph(point_data,outline_points=None, lines=None):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    # Create cubic bounding box to simulate equal aspect ratio
    X = point_data[:,0]
    Y = point_data[:,1]
    Z = point_data[:,2]

    ax.scatter(point_data[:,0], point_data[:,1], point_data[:,2], s=1)

    #bounding box to show foot in scale
    max_range = np.array([X.max()-X.min(), Y.max()-Y.min(), Z.max()-Z.min()]).max()
    Xb = 0.5*max_range*np.mgrid[-1:2:2,-1:2:2,-1:2:2][0].flatten() + 0.5*(X.max()+X.min())
    Yb = 0.5*max_range*np.mgrid[-1:2:2,-1:2:2,-1:2:2][1].flatten() + 0.5*(Y.max()+Y.min())
    Zb = 0.5*max_range*np.mgrid[-1:2:2,-1:2:2,-1:2:2][2].flatten() + 0.5*(Z.max()+Z.min())

    for xb, yb, zb in zip(Xb, Yb, Zb):
        ax.plot([xb], [yb], [zb], 'w')
        
    if (outline_points is not None):
        flatten = outline_points.reshape(no_of_lines*2,3)
        ax.scatter(flatten[:,0], flatten[:,1], flatten[:,2], s=500/no_of_lines, color='red')

    plt.grid()

In [8]:
def project_points(point_array, axis=1, height_limit=100000): #axis values x=0/y=1/z=2 implement limit
    if(axis==0):
        point_array[:,0]=0
    if(axis==1):
        point_array[:,1]=0
    if(axis==2):
        point_array[:,2]=0
        
    return point_array

In [9]:
def get_outline(point_data, no_of_lines = no_of_lines, line_tolarance = line_tolarance):
    
    z_min = min(point_data[:,2])
    z_max =max(point_data[:,2])
    
    step = (z_max-z_min)/(no_of_lines+1)
    
    lines = np.arange(z_min, z_max, step)[1:]
    
    outline_points=[]
    
    for line in lines:
        line_points=[]
        candidates = point_data[np.where(np.logical_and(point_data[:,2]>line, point_data[:,2]<line+line_tolarance))]
        
        if(len(candidates)==0 or len(outline_points)==no_of_lines):
            break
            
        line_points.append(candidates[np.where(candidates[:,0]==min(candidates[:,0]))][0])
        line_points.append(candidates[np.where(candidates[:,0]==max(candidates[:,0]))][0])
        
        
        outline_points.append(line_points)
        
    return np.array(outline_points).reshape((no_of_lines,2,3)),lines

In [10]:
def read_dataset():
    all_set = pd.read_csv("Datasets/all_data.csv", sep=',',header=0)
    train_set=pd.read_csv("Datasets/train_set.csv", sep=',',header=0)
    test_set=pd.read_csv("Datasets/test_set.csv", sep=',',header=0)
    
    return all_set.drop(all_set.index[[0,1]]),train_set.drop(train_set.index[[0,1]]), test_set.drop(test_set.index[[0,1]])

In [11]:
def load_model(model=model_name):
    if(model=="svm"):
        return pickle.load(open("Models/SVM_Classifier.pkcls", 'rb'))
    else:
        return

In [42]:
points_0 = open("Datasets/Foot Models/points/left/3/sub473ll.pts")
point_array_0 = process_data(points_0)
projection_0 = project_points(point_array_0)
outline_0 =get_outline(projection_0)
outline_points_0= outline_0[0]
lines_0=outline_0[1]

In [43]:
points_1 = open("Datasets/Foot Models/points/left/3/sub1673ll.pts")
point_array_1 = process_data(points_1)
projection_1 = project_points(point_array_1)
outline_1 =get_outline(projection_1)
outline_points_1= outline_1[0]
lines_1=outline_1[1]

In [44]:
points_2 = open("Datasets/Foot Models/points/left/3/sub113ll.pts")
point_array_2 = process_data(points_2)
projection_2 = project_points(point_array_2)
outline_2 =get_outline(projection_2)
outline_points_2= outline_2[0]
lines_2=outline_2[1]

In [45]:
plot_graph(projection_0, outline_points=outline_points_0)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [46]:
plot_graph(projection_1,outline_points=outline_points_1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [47]:
plot_graph(projection_2,outline_points=outline_points_2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [48]:
dataset= read_dataset()
all_data= dataset[0]
train_set = dataset[1]
test_set = dataset[2]

In [49]:
model=load_model() #implement classification model
test_features=test_set[["No.","Cluster","Foot Length","Foot Breadth 1","Short Heel Girth"]]

In [104]:
c4_feet = np.array(train_set.loc[train_set["Cluster"]=='C4']["No."])

c4_outline_points = [] 

for foot in c4_feet:
    points = open("Datasets/Foot Models/points/left/3/"+foot+"3ll.pts")
    point_array = process_data(points)
    projection = project_points(point_array)
    outline =get_outline(projection)
    outline_points= outline[0]
    lines=outline[1]
    
    c4_outline_points.append(outline_points)

In [ ]:
# genetic algorithm search for continuous function optimization
from numpy.random import randint
from numpy.random import rand

# objective function to minimize
def objective(x):
    func = 0
    for i in range(0,no_of_lines):
        for j in range(0,2):
            weighted_point = 0
            for k in range(0,len(c4_outline_points)):
                weighted_point = weighted_point + x[k]*c4_outline_points[k][i][j] 
            func = func+ np.linalg.norm(weighted_point-outline_points_0[i][j])
    return func

# decode bitstring to numbers
def decode(bounds, n_bits, bitstring):
    decoded = list()
    largest = 2**n_bits
    for i in range(len(bounds)):
        # extract the substring
        start, end = i * n_bits, (i * n_bits)+n_bits
        substring = bitstring[start:end]
        # convert bitstring to a string of chars
        chars = ''.join([str(s) for s in substring])
        # convert string to integer
        integer = int(chars, 2)
        # scale integer to desired range
        value = bounds[i][0] + (integer/largest) * (bounds[i][1] - bounds[i][0])
        # store
        decoded.append(value)
    return decoded

# tournament selection
def selection(pop, scores, k=3):
    # first random selection
    selection_ix = randint(len(pop))
    for ix in randint(0, len(pop), k-1):
        # check if better (e.g. perform a tournament)
        if scores[ix] < scores[selection_ix]:
            selection_ix = ix
    return pop[selection_ix]

# crossover two parents to create two children
def crossover(p1, p2, r_cross):
    # children are copies of parents by default
    c1, c2 = p1.copy(), p2.copy()
    # check for recombination
    if rand() < r_cross:
        # select crossover point that is not on the end of the string
        pt = randint(1, len(p1)-2)
        # perform crossover
        c1 = p1[:pt] + p2[pt:]
        c2 = p2[:pt] + p1[pt:]
    return [c1, c2]

# mutation operator
def mutation(bitstring, r_mut):
    for i in range(len(bitstring)):
        # check for a mutation
        if rand() < r_mut:
            # flip the bit
            bitstring[i] = 1 - bitstring[i]

# genetic algorithm
def genetic_algorithm(objective, bounds, n_bits, n_iter, n_pop, r_cross, r_mut):
    # initial population of random bitstring
    pop = [randint(0, 2, n_bits*len(bounds)).tolist() for _ in range(n_pop)]
    # keep track of best solution
    best, best_eval = 0, objective(decode(bounds, n_bits, pop[0]))
    # enumerate generations
    for gen in range(n_iter):

        # decode population
        decoded = [decode(bounds, n_bits, p) for p in pop]
        # evaluate all candidates in the population
        scores = [objective(d) for d in decoded]
        # check for new best solution
        for i in range(n_pop):
            if scores[i] < best_eval:
                best, best_eval = pop[i], scores[i]
                print(">%d, new best f(%s) = %f" % (gen,  decoded[i], scores[i]))
        # select parents
        selected = [selection(pop, scores) for _ in range(n_pop)]
        # create the next generation
        children = list()
        for i in range(0, n_pop, 2):
            # get selected parents in pairs
            p1, p2 = selected[i], selected[i+1]
            # crossover and mutation
            for c in crossover(p1, p2, r_cross):
                # mutation
                mutation(c, r_mut)
                # store for next generation
                children.append(c)
        # replace population
        pop = children
    return [best, best_eval]

# define range for input
bounds = [[[0,2]]*len(c4_outline_points)][0]
# define the total iterations
n_iter = 1000
# bits per variable
n_bits = 16
# define the population size
n_pop = 100
# crossover rate
r_cross = 0.9
# mutation rate
r_mut = 1.0 / (float(n_bits) * len(bounds))
# perform the genetic algorithm search
best, score = genetic_algorithm(objective, bounds, n_bits, n_iter, n_pop, r_cross, r_mut)
print('Done!')
decoded = decode(bounds, n_bits, best)
print('f(%s) = %f' % (decoded, score))


#set bound to (-1 to 1) and normalize the foots to max length foot

>0, new best f([0.469879150390625, 0.386993408203125, 0.753753662109375, 0.374053955078125, 1.7967529296875, 0.95318603515625, 0.52581787109375, 1.913177490234375, 0.922882080078125, 0.7196044921875, 0.514434814453125, 0.680877685546875, 0.92083740234375, 1.431793212890625, 1.537109375, 0.247650146484375, 1.90966796875, 0.473175048828125, 1.443756103515625, 0.494354248046875, 1.76165771484375, 0.9078369140625, 0.6885986328125, 1.521636962890625, 1.0977783203125, 0.2943115234375, 0.488739013671875, 1.014617919921875, 0.233245849609375, 0.24481201171875, 0.0943603515625, 1.786651611328125, 1.14605712890625, 1.740264892578125]) = 711664.321228
>0, new best f([0.935546875, 1.06396484375, 0.6016845703125, 1.68646240234375, 1.500335693359375, 1.39923095703125, 1.659515380859375, 0.601470947265625, 0.018402099609375, 1.509490966796875, 0.28302001953125, 0.372344970703125, 0.274261474609375, 0.771881103515625, 0.29669189453125, 1.714996337890625, 0.27972412109375, 0.190826416015625, 1.44830322

In [99]:
decoded

[0.93426513671875, 0.079132080078125]

In [100]:
val=0
for i in range(len(decoded)):
    val=val+c4_outline_points[i]*decoded[i]

In [101]:
plot_graph(projection_0, outline_points=val)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [88]:
val

array([[[-0.66353582,  0.        ,  2.20092656],
        [ 0.05673512,  0.        ,  2.20092656]],

       [[-0.71809118,  0.        ,  2.28885076],
        [ 0.19437382,  0.        ,  2.28885076]],

       [[-0.75612281,  0.        ,  2.37738685],
        [ 0.30287809,  0.        ,  2.37738685]],

       [[-0.78067321,  0.        ,  2.47220496],
        [ 0.40064677,  0.        ,  2.47220496]],

       [[-0.79123097,  0.        ,  2.56243074],
        [ 0.47241189,  0.        ,  2.56687866]],

       [[-0.79194561,  0.        ,  2.64803652],
        [ 0.53552057,  0.        ,  2.65438664]],

       [[-0.79013744,  0.        ,  2.73865275],
        [ 0.60088632,  0.        ,  2.74713755]],

       [[-0.79734823,  0.        ,  2.83473699],
        [ 0.66364493,  0.        ,  2.83751441]],

       [[-0.81070706,  0.        ,  2.92715484],
        [ 0.71934356,  0.        ,  2.92715484]],

       [[-0.82519494,  0.        ,  3.01892489],
        [ 0.77285252,  0.        ,  3.01780183]],



(50, 2, 3)